In [5]:
from pyspark.sql import SparkSession, functions as f, types as t

In [10]:
spark = SparkSession.builder.appName('lect_14_home_task').getOrCreate()

In [18]:
spark.version

'3.3.0'

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

Дампи таблиць знаходяться в папці `data`.
Можете створювати стільки нових клітинок, скільки вам необхідно.

Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)

**Увага!** Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


# init:

In [21]:
import os
BASE_DIR = os.getenv('BASE_DIR')

df_category = spark.read.csv(os.path.join(BASE_DIR, 'category.csv'), inferSchema=True, header=True)
df_film_category = spark.read.csv(os.path.join(BASE_DIR, 'film_category.csv'), inferSchema=True, header=True)
df_inventory = spark.read.csv(os.path.join(BASE_DIR, 'inventory.csv'), inferSchema=True, header=True)
df_film_actor = spark.read.csv(os.path.join(BASE_DIR, 'film_actor.csv'), inferSchema=True, header=True)
df_actor = spark.read.csv(os.path.join(BASE_DIR, 'actor.csv'), inferSchema=True, header=True)
df_rental = spark.read.csv(os.path.join(BASE_DIR, 'rental.csv'), inferSchema=True, header=True)
df_payment = spark.read.csv(os.path.join(BASE_DIR, 'payment.csv'), inferSchema=True, header=True)
df_film = spark.read.csv(os.path.join(BASE_DIR, 'film.csv'), inferSchema=True, header=True)
df_city = spark.read.csv(os.path.join(BASE_DIR, 'city.csv'), inferSchema=True, header=True)
df_address = spark.read.csv(os.path.join(BASE_DIR, 'address.csv'), inferSchema=True, header=True)
df_customer = spark.read.csv(os.path.join(BASE_DIR, 'customer.csv'), inferSchema=True, header=True)


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [23]:
df_film_category.join(df_category, df_category.category_id == df_film_category.category_id, "left") \
     .groupBy('name').count().select(f.col('name').alias('category'),'count') \
     .orderBy(f.desc('count')).show(truncate=False)


+-----------+-----+
|category   |count|
+-----------+-----+
|Sports     |74   |
|Foreign    |73   |
|Family     |69   |
|Documentary|68   |
|Animation  |66   |
|Action     |64   |
|New        |63   |
|Drama      |62   |
|Games      |61   |
|Sci-Fi     |61   |
|Children   |60   |
|Comedy     |58   |
|Travel     |57   |
|Classics   |57   |
|Horror     |56   |
|Music      |51   |
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [43]:
# if you need to pass a set of 10 records to another df, then use .limit(10)
# if just show 10 lines, then .show(10)

df_rental.join(df_inventory, df_rental.inventory_id == df_inventory.inventory_id, "left") \
    .join(df_film_actor, df_inventory.film_id == df_film_actor.film_id, "left") \
    .join(df_actor, df_film_actor.actor_id == df_actor.actor_id, "left") \
    .groupBy('first_name','last_name').count() \
    .select(f.concat('first_name', f.lit(' '), 'last_name').alias('full_name'),'count') \
    .orderBy(f.desc('count')).limit(10).show()


+------------------+-----+
|         full_name|count|
+------------------+-----+
|       SUSAN DAVIS|  825|
|    GINA DEGENERES|  753|
|    MATTHEW CARREY|  678|
|       MARY KEITEL|  674|
|ANGELA WITHERSPOON|  654|
|       WALTER TORN|  640|
|       HENRY BERRY|  612|
|       JAYNE NOLTE|  611|
|        VAL BOLGER|  605|
|     SANDRA KILMER|  604|
+------------------+-----+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [46]:
df_payment.join(df_rental, df_payment.rental_id == df_rental.rental_id, "left") \
    .join(df_inventory, df_rental.inventory_id == df_inventory.inventory_id, "left") \
    .join(df_film_category, df_inventory.film_id == df_film_category.film_id, "left") \
    .join(df_category, df_film_category.category_id == df_category.category_id, "left") \
    .groupBy('name').agg(f.format_number(f.sum('amount'), 2).alias('sum')) \
    .orderBy(f.desc('sum')).show(1)

+------+--------+
|  name|     sum|
+------+--------+
|Sports|5,314.21|
+------+--------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.
Запит має бути без оператора IN

In [29]:
# re-arranged according to the sizes

df_inventory.join(df_film, df_inventory.film_id == df_film.film_id, "right") \
    .filter(df_inventory.film_id.isNull()).select('title').show(truncate=False)

+----------------------+
|title                 |
+----------------------+
|ALICE FANTASIA        |
|APOLLO TEEN           |
|ARGONAUTS TOWN        |
|ARK RIDGEMONT         |
|ARSENIC INDEPENDENCE  |
|BOONDOCK BALLROOM     |
|BUTCH PANTHER         |
|CATCH AMISTAD         |
|CHINATOWN GLADIATOR   |
|CHOCOLATE DUCK        |
|COMMANDMENTS EXPRESS  |
|CROSSING DIVORCE      |
|CROWDS TELEMARK       |
|CRYSTAL BREAKING      |
|DAZED PUNK            |
|DELIVERANCE MULHOLLAND|
|FIREHOUSE VIETNAM     |
|FLOATS GARDEN         |
|FRANKENSTEIN STRANGER |
|GLADIATOR WESTWARD    |
+----------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [47]:
df_film_actor.join(df_film_category, df_film_category.film_id == df_film_actor.film_id, "left") \
    .join(df_actor, ["actor_id"], "left") \
    .join(df_category, df_category.category_id == df_film_category.category_id, "left") \
    .filter(df_category.name == 'Children').groupBy('actor_id', 'first_name', 'last_name') \
    .count().select('first_name', 'last_name', 'count').orderBy(f.col('count').desc(), 'first_name', 'last_name') \
    .limit(3).show()

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|     HELEN|   VOIGHT|    7|
|     KEVIN|  GARLAND|    5|
|      MARY|    TANDY|    5|
+----------+---------+-----+



6.
Вивести міста з кількістю активних та неактивних клієнтів
(в активних customer.active = 1).
Результат відсортувати за кількістю неактивних клієнтів за спаданням.

In [40]:
df_customer.join(df_address, df_customer.address_id == df_address.address_id, "left") \
    .join(df_city, ['city_id'], "left") \
    .groupBy('city_id', 'city').agg(
        f.sum(df_customer.active).alias('sum_active'),
        f.sum(f.when(df_customer.active == 0, 1).otherwise(0)).alias('sum_inactive')
    ).orderBy(f.desc('sum_inactive')).select('city', 'sum_active', 'sum_inactive').show()

+----------------+----------+------------+
|            city|sum_active|sum_inactive|
+----------------+----------+------------+
|       Pingxiang|         0|           1|
|     Szkesfehrvr|         0|           1|
|   Coatzacoalcos|         0|           1|
| Southend-on-Sea|         0|           1|
|          Ktahya|         0|           1|
|         Bat Yam|         0|           1|
|          Amroha|         0|           1|
|          Daxian|         0|           1|
|         Wroclaw|         0|           1|
|      Kumbakonam|         0|           1|
|          Kamyin|         0|           1|
|       Najafabad|         0|           1|
|Charlotte Amalie|         0|           1|
|        Xiangfan|         0|           1|
|        Uluberia|         0|           1|
|          Laredo|         1|           0|
|        Bhilwara|         1|           0|
|        Gulbarga|         1|           0|
|        Songkhla|         1|           0|
|          Mardan|         1|           0|
+----------